# COVID19 - Countries

In [1]:
import os
import ray

In [2]:
import urllib.request
import pandas as pd
import numpy as np
from datetime import datetime,timedelta,date
from tabulate import tabulate
from IPython.display import HTML

In [3]:
# Download data
import get_data as gd
LoadData=True

if LoadData:
    gd.get_data()

# SEAIR-D Model Equations

$$\begin{array}{l}\frac{d s}{d t}=-[\beta i(t) + \beta_2 a(t)-\mu] \cdot s(t)\\ 
\frac{d e}{d t}=[\beta i(t) + \beta_2 a(t)] \cdot s(t) -(\sigma+\mu) \cdot e(t)\\ 
\frac{d a}{d t}=\sigma e(t) \cdot (1-p)-(\gamma+\mu) \cdot a(t) \\
\frac{d i}{d t}=\sigma e(t) \cdot p - (\gamma + \sigma_2 + \sigma_3 + \mu) \cdot i(t)\\ 
\frac{d r}{d t}=(b + \sigma_2) \cdot i(t) + \gamma \cdot a(t) - \mu \cdot r(t)\\
\frac{d k}{d t}=(a + \sigma_3 - \mu) \cdot d(t)
\end{array}$$

The sum of all people accounted is equal to population considered:

$$s + e + a + i + r + k = N$$

The derivate is:

$$\frac{d s}{d t}+\frac{d e}{d t}+\frac{d a}{d t}+\frac{d i}{d t}+\frac{d r}{d t}+\frac{d k}{d t} = 0$$

The last equation does not need to be solve because:

$$\frac{d k}{d t}=-(\frac{d s}{d t}+\frac{d e}{d t}+\frac{d a}{d t}+\frac{d i}{d t}+\frac{d r}{d t})$$

The sum of all rates are equal to zero! The importance of this equation is that it conservates the rates.


## Parameters

N: the population considered

$\beta$: Effective contact rate [1/min]
    
$\gamma$: Recovery(+Mortality) rate $\gamma=(a+b)$ [1/min]

$a$: mortality of healed  [1/min]

$b$: recovery rate  [1/min]

$\sigma$: is the rate at which individuals move from the exposed to the infectious classes. Its reciprocal ($1/\sigma$) is the average latent (exposed) period.

$\sigma_2$: is the rate at which individuals move from the infectious to the healed classes. Its reciprocal ($1/\sigma_2$) is the average latent (exposed) period

$\sigma_3$: is the rate at which individuals move from the infectious to the dead classes. Its reciprocal ($1/\sigma_3$) is the average latent (exposed) period
    
$p$: is the fraction of the exposed which become symptomatic infectious sub-population.

$(1-p)$: is the fraction of the exposed which becomes asymptomatic infectious sub-population.

## Options: Param.CSV file, changeCSV, Adjust Param, Save CSV, Select Specific Country

In [4]:
from IPython.display import display, HTML, Markdown

changeCSV=False
adjustParam=True
paramSave=False
selectCountries=False

## Different Options to Load Param.CSV file

In [5]:
paramOpt=5
model = "YaboxIC"    

if paramOpt==0:
    paramFile="data/param_optimized_Yabox_HistMin-Copy3.csv"
    version2 = "003"

if paramOpt==1:
    paramFile="data/param_optimized_Yabox_HistMin-Copy4.csv"
    version2 = "003"
    
if paramOpt==2:
    paramFile="data/param_optimized_Yabox_HistMin.csv"
    version2 = "004"
    
if paramOpt==3:
    paramFile="data/param_optimized_Yabox_HistMin.csv"
    version2 = "005"

if paramOpt==4:
    paramFile="data/param_optimized_Yabox_HistMin.csv"
    version2 = "006"

if paramOpt==5:
    paramFile="data/param_optimized_Yabox_HistMin.csv"
    version2 = "007"

if paramOpt==6:
    paramFile="data/param_optimized_Yabox_HistMin.csv"
    version2 = "008"

In [6]:
dfparam = pd.read_csv(paramFile)
dfparam = dfparam.dropna()
dfparam.e0=0
dfparam.a0=0

display(Markdown("## Original Initial Parameters"))
display(dfparam)

## Original Initial Parameters

,country,start-date,prediction-range,s0,e0,a0,i0,r0,d0,START,WCASES,WREC,WDTH
0,Brazil,02/29/20,200,8998400,0,0,245,110,523,268,0.9711,0.2139,0.1734
1,China,01/28/20,200,771771134,0,0,200,100,50,50,0.1438,0.0052,0.7554
2,Italy,03/01/20,200,7542132,0,0,735,316,578,287,0.0373,0.5485,0.4752
3,US,02/18/20,200,4922573,0,0,659,903,774,686,0.0118,0.5007,0.5503
4,India,03/12/20,200,670074474,0,0,527,473,492,385,0.0064,0.0212,0.6662


## Modify param.csv file to fit better the data

In [7]:
if changeCSV and (paramOpt==1):
    dfparam['s0'] = pd.to_numeric(dfparam['s0'],errors='coerce')
    dfparam['i0'] = pd.to_numeric(dfparam['i0'],errors='coerce')
    dfparam['d0'] = pd.to_numeric(dfparam['d0'],errors='coerce')
    dfparam['s0'] = dfparam['s0'].astype(float)
    dfparam['i0'] = dfparam['i0'].astype(float)
    dfparam['d0'] = dfparam['d0'].astype(float)
    dfparam.loc[dfparam.country=='Italy','s0'] = dfparam.loc[dfparam.country=='Italy','s0']*1.8
    dfparam.loc[dfparam.country=='China','s0'] = dfparam.loc[dfparam.country=='China','s0']*1.5
    dfparam.loc[dfparam.country=='India','s0'] = dfparam.loc[dfparam.country=='India','s0']*12/100
    dfparam.loc[dfparam.country=='US','s0'] = dfparam.loc[dfparam.country=='US','s0']*8/10
    dfparam['s0'] = pd.to_numeric(dfparam['s0'],errors='coerce')
    dfparam['i0'] = pd.to_numeric(dfparam['i0'],errors='coerce')
    dfparam['d0'] = pd.to_numeric(dfparam['d0'],errors='coerce')
    

# if adjustParam:
#     sCorrect=[1.38,1.0,1.05,1.5,1.5,1.4,1.1,1.5,1.15,1.55,1.35,1.2,1.35,1.15,1.55,1.45,1.25]
#     dfparam.s0=dfparam.s0.multiply(sCorrect[:len(DRS)], axis=0)

if selectCountries and paramOpt==0:
    countries=['Brazil', 
         'US']

if not selectCountries:
    countries=dfparam.country    

dfparam['s0'] = dfparam['s0'].astype(int)
dfparam['i0'] = dfparam['i0'].astype(int)
dfparam['d0'] = dfparam['d0'].astype(int)

if paramSave:
    dfparam.to_csv("new_"+paramFile)    

display(Markdown("## Modified Initial Parameters"))

display(dfparam)

## Modified Initial Parameters

,country,start-date,prediction-range,s0,e0,a0,i0,r0,d0,START,WCASES,WREC,WDTH
0,Brazil,02/29/20,200,8998400,0,0,245,110,523,268,0.9711,0.2139,0.1734
1,China,01/28/20,200,771771134,0,0,200,100,50,50,0.1438,0.0052,0.7554
2,Italy,03/01/20,200,7542132,0,0,735,316,578,287,0.0373,0.5485,0.4752
3,US,02/18/20,200,4922573,0,0,659,903,774,686,0.0118,0.5007,0.5503
4,India,03/12/20,200,670074474,0,0,527,473,492,385,0.0064,0.0212,0.6662


# Solver Loading and Version

In [8]:
%reload_ext autoreload
%autoreload 2
underNotif=True
import LearnerYabox_v5 as Learner


Yabox version:  1.1.0


2020-10-21 14:12:05,778	INFO resource_spec.py:212 -- Starting Ray with 229.98 GiB memory available for workers and up to 72.46 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-10-21 14:12:06,095	INFO services.py:1170 -- View the Ray dashboard at localhost:8265


In [9]:
allCountries=True
cleanRecovered=False
version="2"

if allCountries:
    display(countries)
else:
    countrySelected="Italy"
    display(countrySelected)

0    Brazil
1     China
2     Italy
3        US
4     India
Name: country, dtype: object

In [10]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines){
    return true;}

<IPython.core.display.Javascript object>

# Main Loop

In [ ]:
results=[]
if allCountries:
    for country in countries:
        query = dfparam.query('country == "{}"'.format(country)).reset_index()
        parameters = np.array(query.iloc[:, 2:])[0]
        current = datetime.strptime(parameters[0], '%m/%d/%y')
        parameters[0] = datetime.strftime(current, '%-m/%-d/%y')
        if country=='Italy':
            endDateStr = "10/15/20"
        else:
            endDate = date.today() + timedelta(days = -2)
            endDateStr = datetime.strftime(endDate, '%-m/%-d/%y')
        endDate = datetime.strptime(endDateStr, '%m/%d/%y')
        end_date= datetime.strftime(endDate, '%-m/%-d/%y')
        f=Learner.Learner.remote(country, *parameters, end_date, 
                                 cleanRecovered, version,underNotif=underNotif)
        result = f.train.remote() 
        results.append(result)
else:
    country = countrySelected 
    query = dfparam.query('country == "{}"'.format(country)).reset_index()
    parameters = np.array(query.iloc[:, 2:])[0]
    current = datetime.strptime(parameters[0], '%m/%d/%y')
    parameters[0] = datetime.strftime(current, '%-m/%-d/%y')
    if country=='Italy':
        endDateStr = "9/1/20"
    else:
        endDate = date.today() + timedelta(days = -2)
        endDateStr = datetime.strftime(endDate, '%-m/%-d/%y')
    endDate = datetime.strptime(endDateStr, '%m/%d/%y')
    end_date= datetime.strftime(endDate, '%-m/%-d/%y')  
    f=Learner.Learner.remote(country, *parameters, end_date, 
                             cleanRecovered, version,underNotif=underNotif)
    result = f.train.remote() 
    results.append(result)

# #execute all the queue with max_runner_cap at a time    
results = ray.get(results)

2020-10-21 14:12:06,921	WARNING worker.py:1090 -- WARNING: 3 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


  0%|          | 0/21125000.0 [00:00<?, ?it/s]


2020-10-21 14:12:07,319	WARNING worker.py:1090 -- WARNING: 4 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


  0%|          | 0/21125000.0 [00:00<?, ?it/s]


2020-10-21 14:12:07,725	WARNING worker.py:1090 -- WARNING: 5 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


...

 27%|██▋       | 5808936/21125000.0 [48:20<51:08, 4991.59it/s]
 38%|███▊      | 8110378/21125000.0 [48:22<27:15, 7958.27it/s]
 29%|██▉       | 6130251/21125000.0 [48:21<50:32, 4943.94it/s]
 17%|█▋        | 3649051/21125000.0 [48:21<1:38:32, 2955.55it/s]
 38%|███▊      | 8114406/21125000.0 [48:22<27:21, 7927.50it/s]
 28%|██▊       | 5812345/21125000.0 [48:21<51:37, 4944.05it/s]
 29%|██▉       | 6133753/21125000.0 [48:22<50:37, 4935.08it/s]
 38%|███▊      | 8118435/21125000.0 [48:23<27:33, 7864.96it/s]
 17%|█▋        | 3651753/21125000.0 [48:22<1:37:38, 2982.67it/s]
 28%|██▊       | 5815755/21125000.0 [48:22<51:36, 4944.36it/s]
 18%|█▊        | 3801903/21125000.0 [48:22<9:37:25, 500.01it/s] 
 38%|███▊      | 8122465/21125000.0 [48:23<28:01, 7730.82it/s]
 29%|██▉       | 6137256/21125000.0 [48:22<50:51, 4912.14it/s]
 28%|██▊       | 5819166/21125000.0 [48:22<51:03, 4996.80it/s]
 38%|███▊      | 8126496/21125000.0 [48:24<27:34, 7856.06it/s]
 17%|█▋        | 3654456/21125000.0 [48:23<1:40:3

 39%|███▉      | 8256016/21125000.0 [48:40<26:35, 8066.31it/s]
 18%|█▊        | 3703281/21125000.0 [48:40<1:34:11, 3082.91it/s]
 29%|██▉       | 6218101/21125000.0 [48:39<51:13, 4850.32it/s]
 28%|██▊       | 5901330/21125000.0 [48:39<51:15, 4949.46it/s]
 18%|█▊        | 3860031/21125000.0 [48:39<1:26:48, 3315.03it/s]
 39%|███▉      | 8260080/21125000.0 [48:40<26:03, 8229.97it/s]
 39%|███▉      | 8264145/21125000.0 [48:41<25:57, 8258.69it/s]
 18%|█▊        | 3706003/21125000.0 [48:40<1:34:39, 3067.12it/s]
 29%|██▉       | 6221628/21125000.0 [48:40<51:15, 4846.52it/s]
 28%|██▊       | 5904766/21125000.0 [48:40<51:06, 4963.94it/s]
 18%|█▊        | 3862810/21125000.0 [48:40<1:27:37, 3283.03it/s]
 39%|███▉      | 8268211/21125000.0 [48:41<25:59, 8246.12it/s]
 28%|██▊       | 5908203/21125000.0 [48:40<50:51, 4986.53it/s]
 18%|█▊        | 3708726/21125000.0 [48:41<1:33:20, 3109.77it/s]
 29%|██▉       | 6225156/21125000.0 [48:41<52:31, 4728.24it/s]
 39%|███▉      | 8272278/21125000.0 [48:42<26

 28%|██▊       | 6015246/21125000.0 [49:02<48:21, 5207.66it/s]
 19%|█▊        | 3935415/21125000.0 [49:01<1:23:14, 3441.48it/s]
 40%|███▉      | 8443995/21125000.0 [49:03<26:43, 7907.97it/s]
 18%|█▊        | 3730546/21125000.0 [49:03<2:12:30, 2187.80it/s]
 28%|██▊       | 6018715/21125000.0 [49:02<48:10, 5225.99it/s]
 19%|█▊        | 3938221/21125000.0 [49:02<1:21:34, 3511.20it/s]
 40%|███▉      | 8448105/21125000.0 [49:04<26:42, 7909.88it/s]
 29%|██▊       | 6022185/21125000.0 [49:03<48:50, 5152.88it/s]
 40%|████      | 8452216/21125000.0 [49:04<25:50, 8174.29it/s]
 18%|█▊        | 3733278/21125000.0 [49:04<2:00:25, 2406.96it/s]
 30%|██▉       | 6260491/21125000.0 [49:04<4:58:17, 830.55it/s]
 19%|█▊        | 3941028/21125000.0 [49:03<1:22:30, 3471.14it/s]
 40%|████      | 8456328/21125000.0 [49:05<26:11, 8063.83it/s]
 29%|██▊       | 6025656/21125000.0 [49:04<48:57, 5139.97it/s]
 30%|██▉       | 6264030/21125000.0 [49:04<3:45:05, 1100.39it/s]
 18%|█▊        | 3736011/21125000.0 [49:05

 19%|█▉        | 3997378/21125000.0 [49:20<1:26:31, 3299.17it/s]
 41%|████      | 8596731/21125000.0 [49:22<25:46, 8101.05it/s]
 29%|██▉       | 6112756/21125000.0 [49:21<47:58, 5214.53it/s]
 30%|███       | 6342141/21125000.0 [49:21<52:40, 4676.74it/s]
 18%|█▊        | 3788128/21125000.0 [49:22<1:36:22, 2998.15it/s]
 41%|████      | 8600878/21125000.0 [49:22<25:23, 8222.10it/s]
 19%|█▉        | 4000206/21125000.0 [49:21<1:23:31, 3416.86it/s]
 29%|██▉       | 6116253/21125000.0 [49:22<48:56, 5111.60it/s]
 41%|████      | 8605026/21125000.0 [49:23<25:33, 8162.62it/s]
 30%|███       | 6345703/21125000.0 [49:22<53:14, 4627.19it/s]
 18%|█▊        | 3790881/21125000.0 [49:23<1:38:37, 2929.17it/s]
 41%|████      | 8609175/21125000.0 [49:23<25:08, 8296.54it/s]
 29%|██▉       | 6119751/21125000.0 [49:22<48:17, 5178.71it/s]
 19%|█▉        | 4003035/21125000.0 [49:22<1:24:21, 3383.05it/s]
 30%|███       | 6349266/21125000.0 [49:23<51:47, 4755.54it/s]
 41%|████      | 8613325/21125000.0 [49:24<25

 19%|█▉        | 4065526/21125000.0 [49:40<1:21:54, 3471.09it/s]
 18%|█▊        | 3846151/21125000.0 [49:41<1:38:07, 2935.02it/s]
 30%|███       | 6435078/21125000.0 [49:41<53:18, 4593.19it/s]
 29%|██▉       | 6214575/21125000.0 [49:41<47:59, 5178.50it/s]
 19%|█▉        | 4068378/21125000.0 [49:41<1:21:52, 3472.39it/s]
 18%|█▊        | 3848925/21125000.0 [49:42<1:35:43, 3008.01it/s]
 29%|██▉       | 6218101/21125000.0 [49:41<48:44, 5097.87it/s]
 30%|███       | 6438666/21125000.0 [49:42<52:54, 4626.34it/s]
 19%|█▉        | 4071231/21125000.0 [49:42<1:25:05, 3340.30it/s]
 29%|██▉       | 6221628/21125000.0 [49:42<47:59, 5175.87it/s]
 18%|█▊        | 3851700/21125000.0 [49:43<1:32:31, 3111.74it/s]
 30%|███       | 6442255/21125000.0 [49:43<51:16, 4773.06it/s]
 29%|██▉       | 6225156/21125000.0 [49:43<47:17, 5251.41it/s]
 19%|█▉        | 4074085/21125000.0 [49:42<1:23:07, 3419.04it/s]
 31%|███       | 6445845/21125000.0 [49:43<51:04, 4789.88it/s]
 18%|█▊        | 3854476/21125000.0 [49:4

In [ ]:
ray.shutdown()

# Optimized Parameters and Curves Adjust

In [ ]:
cols=['beta0', 'beta01', 'startT', 'beta2', 'sigma', 'sigma2', 
      'sigma3', 'gamma', 'b', 'gamma2', 'd', 'mu', 'sub', 'subRec', 'subDth']
dfVars = pd.DataFrame(data=results, columns=cols)
if allCountries:
    dfVars['country']=dfparam.country
else:
    dfVars['country']=countrySelected
    display(countrySelected)

In [ ]:
cols=['beta0', 'beta01', 'startT', 'beta2', 'sigma', 'sigma2', 
      'sigma3', 'gamma', 'b', 'gamma2', 'd', 'mu', 'country','sub', 'subRec', 'subDth']
dfVars=dfVars[cols]

display(Markdown("## Under Notification and Adjusts"))
display(dfVars.iloc[:,12:16])

In [ ]:
cols=['country','beta0', 'beta01', 'startT', 'beta2', 'sigma', 'sigma2', 
      'sigma3', 'gamma', 'b', 'gamma2', 'd', 'mu','sub', 'subRec', 'subDth']
dfVars=dfVars[cols]
dfVars.to_csv('./results/underNotif_vars_'+version2+'.csv', sep=",")

display(Markdown("## Simulation Parameters"))

cols=['index','country','$\\beta_0$', '$\\beta_{01}$', 'startT', '$\\beta_2$', '$\\sigma$', '$\\sigma_2$', 
          '$\\sigma_3$', 'a', 'b', 'c', 'd', '$\\mu$','sub', 'subRec', 'subDth']
    
HTML(tabulate(dfVars.iloc[:,:13], headers= cols, floatfmt='.3e', tablefmt='html'))

# Plots

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import pandas as pd
#%load_ext autoreload
%reload_ext autoreload
%autoreload 2
import covid_plots_v2 as cp

In [ ]:
def loadDataFrame(filename):
    df= pd.read_pickle(filename)
    df.columns = [c.lower().replace(' ', '_') for c in df.columns]
    df.columns = [c.lower().replace('(', '') for c in df.columns]
    df.columns = [c.lower().replace(')', '') for c in df.columns]
    return df

## Plot Parameters

In [ ]:
#main country for analysis
countryMain = "Brazil"

#select countries for plotting
countries4Plot=['China',
               'US',
               'India',
                countryMain,
               'Italy'
               ]

#Choose here your options
#opt=0 all plots
#opt=1 corona log plot
#opt=2 logistic model prediction
#opt=3 bar plot with growth rate
#opt=4 log plot + bar plot
#opt=5 SEAIR-D Model
opt = 5

#number of cases to start plotting model in log graph - real data = 100
startCase=100

#all Countries for plotting
countries=dfparam.country

In [ ]:
#do not allow the scrolling of the plots

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines){
    return false;}

## Plot Loop

In [ ]:
#plots one country or all countries  
if opt==5:
    allCountries=True
else:
    allCountries=False
    
if allCountries:
    for country in countries:
        query = dfparam.query('country == "{}"'.format(country)).reset_index()
        startCase = query['START'][0]
        startdate = query['start-date'][0]
        predict_range = query['prediction-range'][0]
        #calcula data máxima dos gráficos
        #200 dias é usado como máximo dos cálculos da derivada das mortes
        lastDate='9/22/20'
        maxDate= datetime.strptime(lastDate, "%m/%d/%y") + timedelta(days = 200) #"2020-08-31"
        maxDateStr = maxDate.strftime("%-m/%-d/%y")
        cp.covid_plots(country, countries4Plot, startdate, predict_range, \
                        startCase, opt, version, show=True, maxDate=maxDateStr, model=model)
else: 
    query = dfparam.query('country == "{}"'.format(country)).reset_index()
    startdate = query['start-date'][0]
    predict_range = query['prediction-range'][0]
    startCase = query['START'][0]
    #calcula data máxima dos gráficos
    #200 dias é usado como máximo dos cálculos da derivada das mortes
    lastDate='9/22/20'
    maxDate= datetime.strptime(lastDate, "%m/%d/%y") + timedelta(days = 200) #"2020-08-31"
    maxDateStr = maxDate.strftime("%-m/%-d/%y")   
    cp.covid_plots(country, countries4Plot,  startdate, predict_range, \
                       startCase, opt, version, show=True, maxDate=maxDateStr, model=model)